<h1><center> A Readily Implemented Atmosphere Sustainability Constraint for Terrestrial Exoplanets Orbiting Magnetically Active Stars </center></h1>

<div style="text-align: justify"> <span style="font-family: Arial; font-size:1.2em;color:black;"> This is an example of how to apply the (magnetic) atmospheric sustainability constraint (mASC) to tidally-locked exoplanets orbiting magnetically active stars.
Kepler-438b is our test-exoplanet here. The following script can be used to find out whether any terrestrial exoplanet of interest, found within the tidal-locked regime of its host-star, could maintain its potential atmosphere in case super-flares in the order of 10^30 - 10^36 erg are observed in the host-star's surface. </div> 

In [1]:
import pandas as pd
import numpy as np
from matplotlib.dates import DayLocator, HourLocator, DateFormatter
from ai import cdas
import scipy
%matplotlib inline
import datetime
import math
import scipy.integrate as integrate
import scipy.special as special
import csv
import numpy as np
import time
from datetime import datetime
from matplotlib.pylab import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import constants as const # everything in SI

In [2]:
# Define the solar (and other) constants

Msun=2.0*10.0**30.0 # kg
Rsun=6.957*10.0**8.0  # m

Mearth=5.972*10.0**24.0 # kg
Rearth=6371.0*10.0**3.0 # m

mu0=4.0*math.pi*10.0**(-7.0) 
mu04pi=mu0/(4.0*math.pi)
sigma_SI=500000.0  # S/m, electrical conductivity
G_cgs=6.67*10.0**(-8.0)   # cgs

In [3]:
# Define the necessary CME parameters

k=0.2 # typical CME aspect ratio
wdeg=20.0  # typical CME half-angular width (deg)
wrad=wdeg*pi/180.0  
rstar=10.0   # (Rstar)
R=k*rstar # flux rope radius (Rstar)
alpha=2.405/R 
H=rstar+R  # CME front height (Rstar)
rmid=H-R  
L=2.0*wrad*rmid  # flux rope length (Rstar)

In [4]:
Efl=[]
LogEfl=[]
LogHmfl=[]
Hm_list=[]
Rextrap_Rstar=[]
Rextr_AU_list=[]
Beq_list=[]
B0_list=[]
Bstar_list=[]
Bratio_list=[]

# Radius & mass of the host-star of interest 
Rstar=0.520*Rsun # (m)
Rstar_cgs=Rstar*100 # (cm)
Mstar=0.544*Msun # (kg)
Mstar_cgs=Mstar*1000.0 # (gr)

# Radius of the exoplanet of interest
# (radius is known from the NASA exoplanet archive)

Rpl_m=1.12*Rearth # (m)
Rpl_cm=Rpl_m*100.0 # (cm)

# But the mass of the exoplanet is unknown;
# that's why we invoke a mass-radius law from Sotin et al., 2007

Mpl_kg=Mearth*(1.12**(1.0/0.306)) # (kg)
Mpl_g=Mpl_kg*1000.0 # (gr)

# Define the necessary constants to calculate later the Bstev scaling law

rc=Rpl_m   # We assume the core radius of the exoplanet is equal to its total radius (m)
core_mass_den=3.0*Mpl_kg/(4.0*math.pi*Rpl_m**3.0)  # the core density of the exoplanet equal to its total density (kg/m3)
d_AU=0.166   # Semi-major axis of the exoplanet (AU)
d_Rstar=0.166*413.46   # Semi-major axis of the exoplanet (Rstar)
d_m=d_Rstar*Rstar   # Semi-major axis of the exoplanet (m)
d_cgs=d_m*100.0   # Semi-major axis of the exoplanet (cm)

P=2.0*math.pi*(d_cgs**3.0/(Mstar_cgs*G_cgs))**0.5  # orbital period of the exoplanet (sec)
print "Kepler's-438b period (sec) = ", P #check if the period is close to the one provided by the NASA exoplanet archive; if yes, all the aforementioned calculations are correct
omega=(2.0*math.pi)/P  # angular self-rotation = angular orbital rotation (rad/s)


# Main 1: Find the near-star CME magnetic field (at 10 Rstar)

n=1.0
i=1.0
m=10.0

Eflare=10.0**30.0

while Eflare <= 10.0**36.0: 
    while n < m**i : 
                    
        Efl.append(Eflare)
        
        LogEflare=np.log10(Eflare)
        LogEfl.append(LogEflare)
        
        LogHmflare=53.4-0.0524*(LogEflare**0.653)*exp(97.45/LogEflare) 
        LogHmfl.append(LogHmflare)
        
        Hm=10.0**LogHmflare
        Hm_list.append(Hm)
        
        J=integrate.quad(lambda x: special.j1(alpha*x)**2*x, 0, R)
        J=J[0]
        
        Bstar=math.sqrt((2.405*Hm)/(4.0*math.pi*L*R*J))/(Rstar_cgs**2.0)
        Bstar_list.append(Bstar)
        
        n=n+m**(i-1.0)
        Eflare=n*10.0**30.0
        
        if Eflare >= 2*10.0**36.0:
            break
    i=i+1.0
    
# Main 2: Extrapolate the near-star CME magnetic field to different radii (up until 215 Rstar)
for rextrapolation in range(10, 215, 1): 
    Rextrap_Rstar.append(rextrapolation)
    Rextr_AU=rextrapolation*0.002418  
    Rextr_AU_list.append(Rextr_AU)  

    for i,value in enumerate(Bstar_list):
            B0=value*(rextrapolation/rstar)**(-1.6)        
            B0_list.append(B0)
            Beq=(2.0**3.0)*B0 
            Beq_list.append(Beq)
            
            DipMom=345153.76*(core_mass_den**0.5)*(omega**0.5)*(rc**3.0)*(sigma_SI**(-0.5))  # Stevenson's scaling law for the dipole magnetic moment (SI)
            Bsclaw=(mu04pi*DipMom/((Rpl_m)**3.0))  # Bstev (SI; Tesla)
            Bsclaw_cgs=Bsclaw*10000.0   # Bstev (cgs; Gauss)
            Bratio=Beq/Bsclaw_cgs # mASC ratio (R-number)
            Bratio_list.append(Bratio)

Kepler's-438b period (sec) =  2885711.47693
